In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### CNN Architecture

In [3]:
class ConvolutionalNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, 3, 1)
        self.conv2 = nn.Conv2d(10, 25, 3, 1)

        self.fc1 = nn.Linear(25*6*6, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(F.max_pool2d(x, 2, 2))
        x = F.relu(F.max_pool2d(x, 2, 2))

        x = x.view(-1, 25*6*6)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [4]:
model = ConvolutionalNeuralNetwork()
model

ConvolutionalNeuralNetwork(
  (conv1): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(10, 25, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=900, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
)

In [5]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

### Dataset Selection

In [6]:
transform = transforms.ToTensor()

In [7]:
train_data = datasets.MNIST(root = 'cnn_data', train = True, download = True, transform = transform)
test_data = datasets.MNIST(root = 'cnn_data', train = False, download = True, transform = transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting cnn_data\MNIST\raw\train-images-idx3-ubyte.gz to cnn_data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting cnn_data\MNIST\raw\train-labels-idx1-ubyte.gz to cnn_data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting cnn_data\MNIST\raw\t10k-images-idx3-ubyte.gz to cnn_data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting cnn_data\MNIST\raw\t10k-labels-idx1-ubyte.gz to cnn_data\MNIST\raw



In [8]:
train_loader = DataLoader(train_data, batch_size = 10, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 10, shuffle = False)

### Training and Evaluation

In [9]:
test_correct = 0

for epoch in range(10):
    total_loss = 0
    for i, (X_train, y_train) in enumerate(train_loader):
        y_prediction = model(X_train)
        loss = loss_function(y_prediction, y_train)
        total_loss += loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'epoch: {epoch + 1}     loss: {total_loss / len(train_loader) :.5f}')

with torch.no_grad():
    for i, (X_test, y_test) in enumerate(test_loader):
        y_value = model(X_test)
        loss = loss_function(y_value, y_test)

        test_correct += (torch.max(y_value, 1)[1] == y_test).sum()

epoch: 1     loss: 0.17247
epoch: 2     loss: 0.12164
epoch: 3     loss: 0.10716
epoch: 4     loss: 0.10311
epoch: 5     loss: 0.10231
epoch: 6     loss: 0.09960
epoch: 7     loss: 0.09501
epoch: 8     loss: 0.09036
epoch: 9     loss: 0.09097
epoch: 10     loss: 0.09090


In [10]:
print(f'Accuracy on the Test Data: {test_correct/len(test_data)*100 :.2f}%')

Accuracy on the Test Data: 96.60%


### Pretrained CNN Model

In [11]:
import torch.optim as optim
from torchvision import models

pretrained_model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

pretrained_model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

pretrained_model.fc = nn.Linear(pretrained_model.fc.in_features, 10)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(pretrained_model.parameters(), lr=0.0001)

for param in pretrained_model.parameters():
    param.requires_grad = False

for param in pretrained_model.fc.parameters():
    param.requires_grad = True

In [12]:
test_correct = 0

for epoch in range(10):
    total_loss = 0
    for i, (X_train, y_train) in enumerate(train_loader):
        y_prediction = pretrained_model(X_train)
        loss = loss_function(y_prediction, y_train)
        total_loss += loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'epoch: {epoch + 1}     loss: {total_loss / len(train_loader) :.5f}')

with torch.no_grad():
    for i, (X_test, y_test) in enumerate(test_loader):
        y_value = pretrained_model(X_test)
        loss = loss_function(y_value, y_test)

        test_correct += (torch.max(y_value, 1)[1] == y_test).sum()

epoch: 1     loss: 1.65961
epoch: 2     loss: 1.41161
epoch: 3     loss: 1.36473
epoch: 4     loss: 1.34735
epoch: 5     loss: 1.32731
epoch: 6     loss: 1.32807
epoch: 7     loss: 1.32147
epoch: 8     loss: 1.31355
epoch: 9     loss: 1.30999
epoch: 10     loss: 1.31081


In [13]:
print(f'Accuracy on the Test Data: {test_correct/len(test_data)*100 :.2f}%')

Accuracy on the Test Data: 57.65%
